<a href="https://colab.research.google.com/github/Aggregate-Intellect/xir/blob/main/manually_entity_retriever.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install beir
!pip install tensorflow-text
!pip install farm-haystack
!pip install --upgrade pip
!pip install git+https://github.com/deepset-ai/haystack.git
!pip install tensorflow
!pip install qwikidata
!pip install pywikibot
!pip install mwparserfromhell

In [2]:
import pandas as pd
from typing import List
import requests
import pandas as pd
from haystack import Document
from haystack.document_stores import FAISSDocumentStore
from haystack.nodes import RAGenerator, DensePassageRetriever, EmbeddingRetriever
from haystack.utils import fetch_archive_from_http, print_answers, print_documents
from haystack.pipelines import DocumentSearchPipeline
from haystack.document_stores import InMemoryDocumentStore

from datasets import load_dataset

from qwikidata.entity import WikidataItem, WikidataLexeme, WikidataProperty
from qwikidata.linked_data_interface import get_entity_dict_from_api

import nltk.data
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [3]:
document_store = FAISSDocumentStore(faiss_index_factory_str="Flat", return_embedding=True)

# Delete existing documents in documents store
document_store.delete_documents()

In [4]:
vali_wikiqa_data = load_dataset(
    'wiki_qa',
    split='train',
)
pd_test_data = pd.DataFrame(vali_wikiqa_data)

pd_test_data = pd.DataFrame(vali_wikiqa_data)

# make sure all sentences ending with .
# if last character is not . ? !, add . at the end
for index in range(0,pd_test_data.shape[0]):
  last_char = pd_test_data['answer'][index][-1]
  if not (last_char =='.' or last_char =='!' or last_char =='?'):
    pd_test_data['answer'][index] = pd_test_data['answer'][index] + '.'


processed_test_wikiqa = pd.DataFrame(columns = ["question_id", "question", "document_title", "document_content", "answer"])
processed_all_wikiqa = pd.DataFrame(columns = ["question_id", "document_title", "document_content"])

for question_id in pd_test_data['question_id'].unique().tolist():
    # print(question_id)
    pd_question = pd_test_data.loc[pd_test_data['question_id'] == question_id]   
    # print(pd_question)
    content = ' '.join(pd_question["answer"].tolist())
    pd_answer = pd_question[pd_question['label'] == 1]
    # print(pd_question["document_title"].iloc[0])
    processed_all_wikiqa = processed_all_wikiqa.append({"question_id": question_id,
                                                        "document_title": pd_question["document_title"].iloc[0],
                                                        "document_content" : content}, ignore_index=True)
    if not pd_answer.empty:
        answer = ' '.join(pd_answer['answer'].tolist())
        new_row = {"question_id": pd_question["question_id"].iloc[0], 
                  "question" : pd_question["question"].iloc[0],
                  "document_title" : pd_question["document_title"].iloc[0],
                  "document_content" : content,
                  "answer": answer}
        processed_test_wikiqa = processed_test_wikiqa.append(new_row, ignore_index=True)
processed_test_wikiqa

Generating test split:   0%|          | 0/6165 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/2733 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/20360 [00:00<?, ? examples/s]

Dataset wiki_qa downloaded and prepared to /root/.cache/huggingface/datasets/wiki_qa/default/0.1.0/d2d236b5cbdc6fbdab45d168b4d678a002e06ddea3525733a24558150585951c. Subsequent calls will reuse this data.


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,question_id,question,document_title,document_content,answer
0,Q1,how are glacier caves formed?,Glacier cave,A partly submerged glacier cave on Perito Moreno Glacier . The ice facade is...,A glacier cave is a cave formed within the ice of a glacier .
1,Q16,how much is 1 tablespoon of water,Tablespoon,This tablespoon has a capacity of about 15 mL. Measuring Spoons. In the US a...,This tablespoon has a capacity of about 15 mL. In the USA one tablespoon (me...
2,Q17,how much are the harry potter movies worth,Harry Potter,Harry Potter is a series of seven fantasy novels written by the British auth...,"The series also originated much tie-in merchandise, making the Harry Potter ..."
3,Q18,how a rocket engine works,Rocket engine,RS-68 being tested at NASA's Stennis Space Center. The nearly transparent ex...,"A rocket engine, or simply ""rocket"", is a jet engine that uses only stored p..."
4,Q21,how are cholera and typhus transmitted and prevented,Cholera,Cholera is an infection in the small intestine caused by the bacterium Vibri...,Transmission occurs primarily by drinking water or eating food that has been...
...,...,...,...,...,...
868,Q3034,what is melissa and joey about,Melissa & Joey,Melissa & Joey is an ABC Family original television series starring Melissa ...,The series follows local politician Mel Burke (Melissa Joan Hart) and Joe Lo...
869,Q3037,What is an economic feature?,Economics,"Economics is the social science that analyzes the production , distribution ...",Other broad distinctions include those between positive economics (describin...
870,Q3039,what is the average american income,Household income in the United States,"US county household median income 2009. Median US household income, 2001-201...","U.S. median household income fell from $51,144 in 2010 to $50,502 in 2011."
871,Q3042,When was Apple Computer founded,Apple Inc.,"Apple Inc., formerly Apple Computer, Inc., is an American multinational corp...","The company was founded on April 1, 1976, and incorporated as Apple Computer..."


In [5]:
docs = []
for index, row in processed_test_wikiqa.iterrows():
    # create haystack document object with text content and doc metadata
    doc = Document(
        content = row['document_content'],
        meta = {
            "document_title": row['document_title']
        }
    )
    docs.append(doc)

document_store.write_documents(docs)
document_store.get_document_count()

Writing Documents:   0%|          | 0/873 [00:00<?, ?it/s]

852

In [6]:
vali_wikiqa_data = load_dataset(
    'wiki_qa',
    split='test',
)
pd_test_data = pd.DataFrame(vali_wikiqa_data)

pd_test_data = pd.DataFrame(vali_wikiqa_data)

# make sure all sentences ending with .
# if last character is not . ? !, add . at the end
for index in range(0,pd_test_data.shape[0]):
  last_char = pd_test_data['answer'][index][-1]
  if not (last_char =='.' or last_char =='!' or last_char =='?'):
    pd_test_data['answer'][index] = pd_test_data['answer'][index] + '.'


processed_test_wikiqa = pd.DataFrame(columns = ["question_id", "question", "document_title", "document_content", "answer"])
processed_all_wikiqa = pd.DataFrame(columns = ["question_id", "document_title", "document_content"])

for question_id in pd_test_data['question_id'].unique().tolist():
    # print(question_id)
    pd_question = pd_test_data.loc[pd_test_data['question_id'] == question_id]   
    # print(pd_question)
    content = ' '.join(pd_question["answer"].tolist())
    pd_answer = pd_question[pd_question['label'] == 1]
    # print(pd_question["document_title"].iloc[0])
    processed_all_wikiqa = processed_all_wikiqa.append({"question_id": question_id,
                                                        "document_title": pd_question["document_title"].iloc[0],
                                                        "document_content" : content}, ignore_index=True)
    if not pd_answer.empty:
        answer = ' '.join(pd_answer['answer'].tolist())
        new_row = {"question_id": pd_question["question_id"].iloc[0], 
                  "question" : pd_question["question"].iloc[0],
                  "document_title" : pd_question["document_title"].iloc[0],
                  "document_content" : content,
                  "answer": answer}
        processed_test_wikiqa = processed_test_wikiqa.append(new_row, ignore_index=True)
processed_test_wikiqa

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,question_id,question,document_title,document_content,answer
0,Q0,HOW AFRICAN AMERICANS WERE IMMIGRATED TO THE US,African immigration to the United States,African immigration to the United States refers to immigrants to the United ...,"As such, African immigrants are to be distinguished from African American pe..."
1,Q4,how a water pump works,Pump,"A small, electrically powered pump. A large, electrically driven pump (elect...","Pumps operate by some mechanism (typically reciprocating or rotary ), and co..."
2,Q20,how old was sue lyon when she made lolita,Lolita (1962 film),Lolita is a 1962 comedy-drama film by Stanley Kubrick based on the classic n...,"The actress who played Lolita, Sue Lyon , was fourteen at the time of filming."
3,Q33,how are antibodies used in,antibody,Each antibody binds to a specific antigen ; an interaction similar to a lock...,"An antibody (Ab), also known as an immunoglobulin (Ig), is a large Y-shaped ..."
4,Q59,HOW MUCH IS CENTAVOS IN MEXICO,Mexican peso,The peso ( sign : $; code : MXN) is the currency of Mexico . Modern peso and...,"The peso is subdivided into 100 centavos, represented by "" ¢ ""."
...,...,...,...,...,...
238,Q2990,what is the main component of vaccines,vaccine,Jonas Salk in 1955 holds two bottles of a culture used to grow polio vaccine...,The agent stimulates the body's immune system to recognize the agent as fore...
239,Q2994,what is preciosa crystal?,Preciosa (corporation),Preciosa is the luxury brand name for the range of precision-cut lead crysta...,Preciosa is the luxury brand name for the range of precision-cut lead crysta...
240,Q3004,who are all of the jonas brothers,Jonas Brothers,"The Jonas Brothers are an American pop rock band. Formed in 2005, they have ...","Formed in 2005, they have gained popularity from the Disney Channel children..."
241,Q3008,who is mary matalin married to,Mary Matalin,"This is about the political professional. For the actress, see Marlee Matlin...",She is married to Democratic political consultant James Carville .


In [7]:
docs = []
for index, row in processed_test_wikiqa.iterrows():
    # create haystack document object with text content and doc metadata
    doc = Document(
        content = row['document_content'],
        meta = {
            "document_title": row['document_title']
        }
    )
    docs.append(doc)

document_store.write_documents(docs)
document_store.get_document_count()


Writing Documents:   0%|          | 0/243 [00:00<?, ?it/s]

1081

In [8]:
from haystack.retriever.dense import EmbeddingRetriever

retriever = EmbeddingRetriever(
   document_store=document_store,
   embedding_model="flax-sentence-embeddings/all_datasets_v3_mpnet-base",
   model_format="sentence_transformers"
)

document_store.update_embeddings(retriever, batch_size=128)

Updating Embedding:   0%|          | 0/1081 [00:00<?, ? docs/s]

Batches:   0%|          | 0/4 [00:00<?, ?it/s]

Batches:   0%|          | 0/4 [00:00<?, ?it/s]

Batches:   0%|          | 0/4 [00:00<?, ?it/s]

Batches:   0%|          | 0/4 [00:00<?, ?it/s]

Batches:   0%|          | 0/4 [00:00<?, ?it/s]

Batches:   0%|          | 0/4 [00:00<?, ?it/s]

Batches:   0%|          | 0/4 [00:00<?, ?it/s]

Batches:   0%|          | 0/4 [00:00<?, ?it/s]

Batches:   0%|          | 0/2 [00:00<?, ?it/s]

In [9]:
from haystack.pipelines import DocumentSearchPipeline
pipeline = DocumentSearchPipeline(retriever = retriever)
document_store.get_embedding_count()

1081

In [17]:
import pandas as pd

pd_test_data = pd.read_csv("/content/entity&relationship_extraction.csv")

In [18]:
#df_direct_answer = pd_test_data[pd_test_data['answer by entities and relationships'].notna()]
cols = ['answer by entities and relationships', 'entities']
df_direct_answer = pd_test_data[pd_test_data[cols].notnull().any(axis=1)]
df_direct_answer

,question,answer by entities and relationships,entities_name,entities,relationships
0,HOW AFRICAN AMERICANS WERE IMMIGRATED TO THE US,NaN,"African_Americans, immigration, United_States","Q49085, Q131288, Q30",[]
1,how a water pump works,NaN,"Water pump, work","Q2245442, Q42213",[]
2,how old was sue lyon when she made lolita,NaN,"Sue Lyon, Lolita","Q436722, Q536560",[]
3,how are antibodies used in,NaN,"antibody, use","Q79460, Q1724915",[]
4,HOW MUCH IS CENTAVOS IN MEXICO,NaN,Mexican centavo,Q4730,[]
...,...,...,...,...,...
238,what is the main component of vaccines,NaN,vaccine,Q134808,[]
239,what is preciosa crystal?,NaN,Presciosa,Q3502967,[]
240,who are all of the jonas brothers,"Kevin Jonas, Joe Jonas, Nick Jonas",NaN,Q134233,"[['has parts', 'P527']]"
241,who is mary matalin married to,James Carville,NaN,Mary Matalin (American political consultant) / spouse,NaN


# query expansion using knowledge graph

In [19]:
def query_expansion(original_query):
  pd_question = df_direct_answer.loc[df_direct_answer['question'] == original_query] 
  
  if not pd_question.empty: 
    
    # first
    if str(pd_question['answer by entities and relationships'].iloc[0]) != 'nan':
      new_query = original_query + " " + pd_question['answer by entities and relationships'].iloc[0]
      return new_query
    
    # second
    elif str(pd_question['entities'].iloc[0]) != 'nan' and len(pd_question['entities'].iloc[0].split(", ")) == 1:
      # entity_id_list = pd_question['entities'].iloc[0].split(", ")
      entity_id = pd_question['entities'].iloc[0]
      entity_dict = get_entity_dict_from_api(entity_id)
      entity_dict = WikidataItem(entity_dict)
      description = entity_dict.get_description()
      new_query = original_query + " " + description
      return new_query
    
    else:
      if str(pd_question['entities_name'].iloc[0]) != 'nan':
        new_query = original_query + str(pd_question['entities_name'].iloc[0])
        return new_query
      
  return original_query

In [25]:
import json
import requests

MY_GCUBE_TOKEN = "f83a241f-c321-4473-ae3c-0dd2151382fd-843339462"

class WATAnnotation:
    # An entity annotated by WAT

    def __init__(self, d):

        # char offset (included)
        self.start = d['start']
        # char offset (not included)
        self.end = d['end']

        # annotation accuracy
        self.rho = d['rho']
        # spot-entity probability
        self.prior_prob = d['explanation']['prior_explanation']['entity_mention_probability']

        # annotated text
        self.spot = d['spot']

        # Wikpedia entity info
        self.wiki_id = d['id']
        self.wiki_title = d['title']


    def json_dict(self):
        # Simple dictionary representation
        return {'wiki_title': self.wiki_title,
                'wiki_id': self.wiki_id,
                'start': self.start,
                'end': self.end,
                'rho': self.rho,
                'prior_prob': self.prior_prob
                }


def wat_entity_linking(text):
    # Main method, text annotation with WAT entity linking system
    wat_url = 'https://wat.d4science.org/wat/tag/tag'
    payload = [("gcube-token", MY_GCUBE_TOKEN),
               ("text", text),
               ("lang", 'en'),
               ("tokenizer", "nlp4j"),
               ('debug', 9),
               ("method",
                "spotter:includeUserHint=true:includeNamedEntity=true:includeNounPhrase=true,prior:k=50,filter-valid,centroid:rescore=true,topk:k=5,voting:relatedness=lm,ranker:model=0046.model,confidence:model=pruner-wiki.linear")]

    response = requests.get(wat_url, params=payload)
    return [WATAnnotation(a) for a in response.json()['annotations']]

def print_wat_annotations(wat_annotations):
    json_list = [w.json_dict() for w in wat_annotations]
    print(json.dumps(json_list, indent=4))

In [36]:
import urllib
import json
import requests
from qwikidata.entity import WikidataItem, WikidataLexeme, WikidataProperty
from qwikidata.linked_data_interface import get_entity_dict_from_api

def query_to_description(original_question):
    wat_annotations = wat_entity_linking(original_question)
    json_list = [w.json_dict() for w in wat_annotations]
    print(json_list)
    if json_list != []:
      if len(json_list) == 1:
          if "wiki_title" in json_list[0].keys():
              wiki_title = json_list[0]["wiki_title"]
              url = "https://en.wikipedia.org/w/api.php?action=query&prop=pageprops&ppprop=wikibase_item&redirects=1&titles=" + wiki_title + "&format=json"
              #url = "https://en.wikipedia.org/w/api.php?action=query&prop=wikitext&ppprop=wikibase_item&redirects=1&titles=" + wiki_title
              print(url)
              r = requests.get(url)
              data = json.loads(r.text)
              data = json.dumps(data)
              # extract wikidata entity id
              data = data.partition("wikibase_item\": \"")[2]
              entity_id = data.partition("\"")[0]
              print("entity id: ", entity_id)
              entity_dict = get_entity_dict_from_api(entity_id)
              entity_dict = WikidataItem(entity_dict)
              description = entity_dict.get_description()
              if description != None:
                  print("Description: ", description)
                  return description
    return None

# Accuracy with original query

In [ ]:
from haystack.document_stores import InMemoryDocumentStore

number_total = 0
number_correct = 0
number_correctly_retrieved_passage = 0

for index, row in processed_test_wikiqa.iterrows():
  query = row['question']
  #if query != new_query:
  number_total += 1
  print("Original Query: ", query)

  result = pipeline.run(
            query=query, 
            params={"Retriever": {"top_k": 1}}
            )
  
  retrieved_document = result["documents"][0].content
  target_document = processed_test_wikiqa['document_content'][index]
  # print("query: ", query)
  
  if retrieved_document == target_document:
      number_correctly_retrieved_passage += 1


      sentence_list = nltk.tokenize.sent_tokenize(row['document_content'])
      
      sentence_document_store = InMemoryDocumentStore()
      sentence_document_store.delete_documents()
      
      docs = []
      for sentence in sentence_list:
        doc = Document(content=sentence,
                      meta={"article_title": row['document_title']})
        docs.append(doc)
      sentence_document_store.write_documents(docs)
      
      sentence_retriever = EmbeddingRetriever(
            document_store=sentence_document_store,
            embedding_model="flax-sentence-embeddings/all_datasets_v3_mpnet-base",
            model_format="sentence_transformers")
      
      sentence_document_store.update_embeddings(sentence_retriever)
      sentence_pipeline = DocumentSearchPipeline(retriever = sentence_retriever)
      sentence_result = sentence_pipeline.run(query=query, 
                                              params={"Retriever": {"top_k": 1}})
      retrieved_sentence = sentence_result["documents"][0].content
      target_sentence = processed_test_wikiqa['answer'][index]
      if str(retrieved_sentence) in " ".join(str(target_sentence).split()):
          number_correct += 1
          print("!!!!!!!!!")
          print("Found")
      else:
          print("Not Found")

In [24]:
print(number_total)
print(number_correctly_retrieved_passage / number_total)
print(number_correct / number_correctly_retrieved_passage)

243
0.9753086419753086
0.6666666666666666


# Caculate accuracy with query expansion (WAT)

In [ ]:
from haystack.document_stores import InMemoryDocumentStore

number_total = 0
number_correct = 0
number_correctly_retrieved_passage = 0

for index, row in processed_test_wikiqa.iterrows():
  query = row['question']
  #if query != new_query:
  number_total += 1

  entity_description = query_to_description(query)
  if entity_description != None and entity_description != "":
      # query = entity_description
      new_query = query + " " + entity_description
      print("query: ", query)
  else:
    new_query = query


  result = pipeline.run(
            query=new_query, 
            params={"Retriever": {"top_k": 1}}
            )
  
  retrieved_document = result["documents"][0].content
  target_document = processed_test_wikiqa['document_content'][index]
  # print("query: ", query)
  
  if retrieved_document == target_document:
      number_correctly_retrieved_passage += 1


      sentence_list = nltk.tokenize.sent_tokenize(row['document_content'])
      
      sentence_document_store = InMemoryDocumentStore()
      sentence_document_store.delete_documents()
      
      docs = []
      for sentence in sentence_list:
        doc = Document(content=sentence,
                      meta={"article_title": row['document_title']})
        docs.append(doc)
      sentence_document_store.write_documents(docs)
      
      sentence_retriever = EmbeddingRetriever(
            document_store=sentence_document_store,
            embedding_model="flax-sentence-embeddings/all_datasets_v3_mpnet-base",
            model_format="sentence_transformers")
      
      sentence_document_store.update_embeddings(sentence_retriever)
      sentence_pipeline = DocumentSearchPipeline(retriever = sentence_retriever)
      sentence_result = sentence_pipeline.run(query=new_query, 
                                              params={"Retriever": {"top_k": 1}})
      retrieved_sentence = sentence_result["documents"][0].content
      target_sentence = processed_test_wikiqa['answer'][index]
      if str(retrieved_sentence) in " ".join(str(target_sentence).split()):
          number_correct += 1
          print("!!!!!!!!!")
          print("Found")
      else:
          print("Not Found")

In [38]:
print(number_total)
print(number_correctly_retrieved_passage / number_total)
print(number_correct / number_correctly_retrieved_passage)

243
0.9711934156378601
0.6271186440677966


# Calculate accuracy with query expansion （labeling manually）

In [ ]:
from haystack.document_stores import InMemoryDocumentStore

number_total = 0
number_correct = 0
number_correctly_retrieved_passage = 0

for index, row in processed_test_wikiqa.iterrows():
  query = row['question']
  new_query = query_expansion(query)
  #if query != new_query:
  number_total += 1
  print("Original Query: ", query)
  print("New Query: ", new_query)

  result = pipeline.run(
            query=new_query, 
            params={"Retriever": {"top_k": 1}}
            )
  
  retrieved_document = result["documents"][0].content
  target_document = processed_test_wikiqa['document_content'][index]
  # print("query: ", query)
  
  if retrieved_document == target_document:
      number_correctly_retrieved_passage += 1
      sentence_list = nltk.tokenize.sent_tokenize(row['document_content'])
      
      sentence_document_store = InMemoryDocumentStore()
      sentence_document_store.delete_documents()
      
      docs = []
      for sentence in sentence_list:
        doc = Document(content=sentence,
                      meta={"article_title": row['document_title']})
        docs.append(doc)
      sentence_document_store.write_documents(docs)
      
      sentence_retriever = EmbeddingRetriever(
            document_store=sentence_document_store,
            embedding_model="flax-sentence-embeddings/all_datasets_v3_mpnet-base",
            model_format="sentence_transformers")
      
      sentence_document_store.update_embeddings(sentence_retriever)
      sentence_pipeline = DocumentSearchPipeline(retriever = sentence_retriever)
      sentence_result = sentence_pipeline.run(query=new_query, 
                                              params={"Retriever": {"top_k": 1}})
      retrieved_sentence = sentence_result["documents"][0].content
      target_sentence = processed_test_wikiqa['answer'][index]
      if str(retrieved_sentence) in " ".join(str(target_sentence).split()):
          number_correct += 1
          print("!!!!!!!!!")
          print("Found")
      else:
          print("Not Found")

In [28]:
print(number_total)
print(number_correctly_retrieved_passage / number_total)
print(number_correct / number_correctly_retrieved_passage)

243
0.9794238683127572
0.7100840336134454
